In [41]:
from matplotlib import pyplot as plt
import numpy as np
import json
import requests
import pandas as pd
import time
import keras
from keras.layers import Dropout, Dense, Activation, LSTM, Input, Embedding
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

from datetime import datetime  
from datetime import timedelta  

In [56]:
def line_plot(line1, line2, label1=None, label2=None, title=''):
    fig, ax = plt.subplots(1, figsize=(25, 15))
    ax.plot(line1, label=label1, linewidth=2)
    ax.plot(line2, label=label2, linewidth=2)
    
    for i, j in zip(line1, line2):
        ax.plot([], [],)
    
    ax.set_ylabel('price [USD]', fontsize=14)
    ax.set_title(title, fontsize=18)
    ax.legend(loc='best', fontsize=18)
    plt.show()
    
def create_model(input_shape, output_neurons=1):
    model = Sequential()
    model.add(LSTM(20, input_shape=(input_shape[1], input_shape[2])))
    model.add(Dropout(0.5))
    model.add(Dense(output_neurons, activation=keras.activations.linear))

    model.compile(loss=keras.losses.mse, optimizer='nadam')

    return model


def normalize_data(data):
    return normalize(data, norm='l2', axis=1, copy=True)

def download_data():
    endpoint = 'https://min-api.cryptocompare.com/data/histoday'
    res = requests.get(endpoint + '?fsym='+coin+'&tsym=USD&limit=2000')
    hist = pd.DataFrame(json.loads(res.content)['Data'])
    hist = hist.set_index('time')
    hist.index = pd.to_datetime(hist.index, unit='s')
    hist.head()
    return hist

In [65]:
coin = 'BTC'
data = download_data()
y_data = data['close']
x_data = data.drop(['close'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, shuffle=False)

x_train = np.reshape(x_train.as_matrix(), (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test.as_matrix(), (x_test.shape[0], 1, x_test.shape[1]))
model = create_model(x_train.shape, output_neurons=1)

history = model.fit(x=x_train,
          y=y_train,
          epochs=100,
          batch_size=128,
          validation_data=(x_test, y_test),
          verbose=0)
print(history.history['val_loss'][-1])




83197945.19402985


(2001, 6)